# 로이터 뉴스 데이터로 다항분류  
### 레이블이 46개의 범주로 구성되어있음

    이전까지는 독립변수를 두고 변수가 여러개 있을때 독립변수를 가지고 분류를 하거나 예측을 했다

    그러나 이제는 독립변수들이 문장(자연어)으로 되어있음 그래서 이걸 가지고 RNN을 하는 것

    게시판 글이 들어왓다 이게 스팸성 글이야 그런데 이 안의 내용중 안좋은 글일때 게시판 글이 아닐수 있도록 차단이 가능하다
    분류를 통해 LSTM이 될수 있다. LSTM으로 소스를 만든다.새로운 소스로 바꾸어주면 됨


In [ ]:
import numpy as np
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
#레이블에서 원핫처리 안하고 사용하기

(x_train, y_train),(x_test,y_test) = reuters.load_data(test_split=0.3, num_words=1000) # 7대 3으로 나눔, 숫자 제한 1000
print(x_train.shape, y_train.shape,x_test.shape,y_test.shape) #(7859,) (7859,) (3369,) (3369,)
print(x_train[:1])
print(y_train[:1])
category= np.max(y_train)+1
print('category : ',category)
print()

word_index=reuters.get_word_index()
print(word_index)

index_to_word = {}
for k, v in word_index.items():
    index_to_word[v]=k
print(index_to_word)
print(index_to_word[1]) # the가 현재 corpus에서 가장 많이 등장하는 단어, 123번째에 해당하는 건 sum
print(index_to_word[123])
print(' '.join([index_to_word[i] for i in x_train[0]]))
# [list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, .... 은
# 사실은 the of of mln loss for plc said at only ended said of could 1 t 이런식으로 구성된 것

import matplotlib.pyplot as plt
# plt.hist([len(i) for i in x_train],bins=30)
# plt.xlabel('length')
# plt.ylabel('number')
# plt.show()

In [ ]:
# feature 데이터의 크기를 통일
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)
print(x_train[:1])


#model
model = Sequential()
model.add(Embedding(1000, 100)) #embedding이 됨 - 특정차원(100)으로 입력벡터를 주입 후 학습을 통해 가중치 - 기울기 단어 사이의 관계가 반영되지는 않는다. word2vec 단어와 단어 사이의 관계가 반영되지 않는다.- word2vec과는틀림
model.add(LSTM(128,activation='tanh'))
model.add(Dense(46, activation='softmax'))
print(model.summary())

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history=model.fit(x_train, y_train, batch_size=64, epochs=30, validation_split=0.2, verbose=2)
#데이터의 크기도 중요하지만 내 컴퓨터의 메모리 크기도 중요함

print('eval acc: %.3f'%(model.evaluate(x_test,y_test)[1])) #many to one으로 나가는 것

In [ ]:
# 시각화
vloss= history.history['val_loss']
loss=history.history['loss']
epoch = np.arange(len(loss))
plt.plot(epoch, vloss, marker='.',c='red',label='val_loss')
plt.plot(epoch, loss, marker='s',c='blue',label='loss')
plt.legend()

plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

#전통적인 방법 LSTM빼고 Dense만 써주어야 한다.